In [14]:
#!/usr/bin/env python3
"""
Simple Single-Agent Demo: planning, strategy, autonomy
- Input: a natural-language goal
- The agent:
  1) Plans: decomposes the goal into actionable steps
  2) Strategizes: selects channels/tactics based on constraints (budget, time)
  3) Acts autonomously: iterates, fills missing info via tools, and produces an output artifact
"""

from __future__ import annotations
import re
import math
import random
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

In [15]:
# -----------------------------
# "Tools" the agent can use
# -----------------------------

class CalculatorTool:
    """Tiny safe-ish calculator for simple expressions like '50000*0.25'."""
    ALLOWED = set("0123456789.+-*/()% ")

    def run(self, expr: str) -> float:
        if not set(expr) <= self.ALLOWED:
            raise ValueError("Unsafe expression.")
        # eval with limited globals/locals
        return float(eval(expr, {"__builtins__": None}, {}))

class KnowledgeBaseTool:
    """
    Mocked 'research' tool with pragmatic defaults the agent can consult.
    In real life, swap with APIs, DBs, or LLM calls.
    """
    KB = {
        "cpc_india_sports_paid_social": 6.5,       # ₹ per click (rough heuristic)
        "ctr_organic_social": 0.015,               # 1.5% click-through
        "conv_rate_ecom": 0.02,                    # 2% site conversion
        "avg_order_value": 1800.0,                 # ₹
        "influencer_micro_cost_per_post": 5000.0,  # ₹
        "email_ctr": 0.03,
        "email_conv": 0.02,
        "best_posting_times": ["7:30 AM", "12:30 PM", "7:30 PM"],
        "channels": ["Paid Social", "Organic Social", "Influencers", "Email/SMS", "Content/Blog"],
    }

    def lookup(self, key: str) -> Any:
        return self.KB.get(key)

class NoteTool:
    """Simple persistent memory for the run."""
    def __init__(self):
        self.memory: Dict[str, Any] = {}

    def set(self, k: str, v: Any):
        self.memory[k] = v

    def get(self, k: str, default=None):
        return self.memory.get(k, default)

    def add_list(self, k: str, v: Any):
        self.memory.setdefault(k, []).append(v)

class SchedulerTool:
    """Creates a day-by-day schedule and checklist."""
    def make_schedule(self, days: int, slots: Optional[List[str]] = None) -> List[Dict[str, Any]]:
        if slots is None:
            slots = ["Morning", "Afternoon", "Evening"]
        plan = []
        for d in range(1, days + 1):
            plan.append({
                "day": d,
                "slots": {s: [] for s in slots}
            })
        return plan

class IdeaTool:
    """Lightweight idea generator with deterministic seeds."""
    def brainstorm_posts(self, brand: str, theme: str, n: int = 5) -> List[str]:
        random.seed(42 + len(brand) + len(theme))
        verbs = ["Crush", "Chase", "Own", "Level-Up", "Ignite", "Unlock"]
        angles = ["form tips", "micro-workouts", "gear care", "coach wisdom", "community stories", "before/after"]
        ideas = []
        for _ in range(n):
            v = random.choice(verbs)
            a = random.choice(angles)
            ideas.append(f"{v} your game: {a} ({theme} × {brand})")
        return list(dict.fromkeys(ideas))[:n]


In [16]:
# -----------------------------
# Planner & Strategy Modules
# -----------------------------

@dataclass
class Step:
    name: str
    done: bool = False
    details: Dict[str, Any] = field(default_factory=dict)

class Planner:
    """Turns a goal into steps; extracts constraints like budget/days."""
    BUDGET_REGEX = r"(₹|INR|\$)?\s?([0-9][0-9,\.]*)"
    DAYS_REGEX = r"(\d+)\s*(day|days|d)\b"

    def parse_budget(self, text: str) -> Optional[float]:
        # Pick the largest number as likely budget
        matches = re.findall(self.BUDGET_REGEX, text, flags=re.I)
        nums = []
        for _, raw in matches:
            try:
                nums.append(float(raw.replace(",", "")))
            except:
                pass
        return max(nums) if nums else None

    def parse_days(self, text: str) -> Optional[int]:
        m = re.search(self.DAYS_REGEX, text, flags=re.I)
        return int(m.group(1)) if m else None

    def plan(self, goal: str) -> Dict[str, Any]:
        budget = self.parse_budget(goal)
        days = self.parse_days(goal)
        steps = [
            Step("Clarify constraints", details={"need": ["budget", "days"]}),
            Step("Choose channels and split budget"),
            Step("Create content ideas"),
            Step("Build schedule & checklist"),
            Step("Forecast outcomes"),
            Step("Assemble final brief"),
        ]
        return {"budget": budget, "days": days, "steps": steps}

class Strategist:
    """Allocates budget, picks channels based on constraints + KB."""
    def __init__(self, kb: KnowledgeBaseTool, calc: CalculatorTool):
        self.kb = kb
        self.calc = calc

    def choose_channels(self, budget: float, days: int) -> Dict[str, Any]:
        # Simple rules:
        # - Small budget → emphasize organic + micro-influencers
        # - Medium+ → include paid social
        alloc = {}
        if budget < 20000:
            alloc = {"Organic Social": 0.0, "Influencers": 0.6, "Content/Blog": 0.2, "Email/SMS": 0.2}
        elif budget < 100000:
            alloc = {"Paid Social": 0.5, "Influencers": 0.2, "Organic Social": 0.15, "Email/SMS": 0.15}
        else:
            alloc = {"Paid Social": 0.6, "Influencers": 0.2, "Content/Blog": 0.1, "Email/SMS": 0.1}
        # Round allocations and compute rupees
        split = {k: round(v, 2) for k, v in alloc.items()}
        rupees = {k: round(budget * v) for k, v in split.items()}
        return {"allocation": split, "rupees": rupees, "days": days}

    def simple_forecast(self, rupees: Dict[str, float]) -> Dict[str, Any]:
        # Ballpark forecast using KB heuristics (very rough, just to show reasoning)
        cpc = self.kb.lookup("cpc_india_sports_paid_social")
        conv = self.kb.lookup("conv_rate_ecom")
        aov = self.kb.lookup("avg_order_value")
        influencer_cost = self.kb.lookup("influencer_micro_cost_per_post")

        results = {}

        # Paid Social
        if "Paid Social" in rupees and rupees["Paid Social"] > 0 and cpc:
            clicks = rupees["Paid Social"] / cpc
            orders = clicks * conv
            rev = orders * aov
            results["Paid Social"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        # Influencers
        if "Influencers" in rupees and rupees["Influencers"] > 0 and influencer_cost:
            posts = max(1, int(rupees["Influencers"] // influencer_cost))
            # Assume each micro post yields ~150 clicks, 2% convert
            clicks = posts * 150
            orders = clicks * conv
            rev = orders * aov
            results["Influencers"] = {"posts": posts, "clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        # Organic & Email: assume minimal direct spend
        if "Organic Social" in rupees:
            clicks = 400  # placeholder
            orders = clicks * conv
            results["Organic Social"] = {"clicks": clicks, "orders": int(orders)}

        if "Email/SMS" in rupees:
            # Assume list of 10k; 3% CTR; 2% convert
            list_size = 10000
            ctr = self.kb.lookup("email_ctr")
            econv = self.kb.lookup("email_conv")
            clicks = list_size * ctr
            orders = clicks * econv
            rev = orders * aov
            results["Email/SMS"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        # Totals
        totals = {
            "orders": sum(v.get("orders", 0) for v in results.values()),
            "revenue": sum(v.get("revenue", 0) for v in results.values())
        }
        results["TOTALS"] = totals
        return results


In [17]:
#-----------------------------
# The Agent
# -----------------------------

@dataclass
class AgentOutput:
    goal: str
    constraints: Dict[str, Any]
    strategy: Dict[str, Any]
    schedule: List[Dict[str, Any]]
    ideas: List[str]
    forecast: Dict[str, Any]
    brief: str

class SingleAgent:
    def __init__(self):
        self.calc = CalculatorTool()
        self.kb = KnowledgeBaseTool()
        self.notes = NoteTool()
        self.scheduler = SchedulerTool()
        self.ideas = IdeaTool()
        self.planner = Planner()
        self.strategist = Strategist(self.kb, self.calc)

    # ---- Autonomy loop ----
    def run(self, goal: str, brand: str = "YourSportsBrand", theme: str = "Performance") -> AgentOutput:
        plan = self.planner.plan(goal)
        budget, days = plan["budget"], plan["days"]

        # Step 1: Clarify constraints (self-serve defaults if missing)
        if budget is None:
            budget = 50000.0  # sensible default for demo
        if days is None:
            days = 3

        self.notes.set("constraints", {"budget": budget, "days": days})

        # Step 2: Choose channels + split budget
        strat = self.strategist.choose_channels(budget, days)
        self.notes.set("strategy", strat)

        # Step 3: Generate content ideas
        content_ideas = self.ideas.brainstorm_posts(brand, theme, n=6)
        self.notes.set("ideas", content_ideas)

        # Step 4: Build schedule (fillable slots)
        sched = self.scheduler.make_schedule(days, slots=self.kb.lookup("best_posting_times"))
        # Fill schedule with actions derived from strategy
        for day in sched:
            for t in day["slots"]:
                day["slots"][t] = [
                    f"Post: {random.choice(content_ideas)}",
                    "Story/Reel: behind-the-scenes",
                ]
        self.notes.set("schedule", sched)

        # Step 5: Forecast outcomes
        forecast = self.strategist.simple_forecast(strat["rupees"])
        self.notes.set("forecast", forecast)

        # Step 6: Final brief (assembled autonomously)
        brief = self._assemble_brief(goal, brand, theme, strat, sched, content_ideas, forecast)

        return AgentOutput(
            goal=goal,
            constraints=self.notes.get("constraints"),
            strategy=strat,
            schedule=sched,
            ideas=content_ideas,
            forecast=forecast,
            brief=brief
        )

    # ---- Helper to render a human-friendly brief ----
    def _assemble_brief(self, goal, brand, theme, strat, sched, ideas, forecast) -> str:
        lines = []
        c = self.notes.get("constraints")
        lines.append(f"# Launch Brief\n")
        lines.append(f"**Goal:** {goal}")
        lines.append(f"**Brand/Theme:** {brand} / {theme}")
        lines.append(f"**Constraints:** Budget ₹{int(c['budget'])}, Duration {c['days']} days\n")

        lines.append("## Channel Strategy & Budget Split")
        for k, v in strat["allocation"].items():
            lines.append(f"- {k}: {int(v*100)}% (₹{strat['rupees'][k]})")

        lines.append("\n## Content Ideas (shortlist)")
        for i in ideas:
            lines.append(f"- {i}")

        lines.append("\n## Day-by-Day Schedule")
        for day in sched:
            lines.append(f"- **Day {day['day']}**")
            for slot, tasks in day["slots"].items():
                for task in tasks:
                    lines.append(f"  - {slot}: {task}")

        lines.append("\n## Rough Forecast (very rough, for planning only)")
        for ch, vals in forecast.items():
            if ch == "TOTALS":
                continue
            pretty = ", ".join(f"{k}={v}" for k, v in vals.items())
            lines.append(f"- {ch}: {pretty}")
        totals = forecast.get("TOTALS", {})
        lines.append(f"- **TOTALS:** orders={totals.get('orders', 0)}, revenue=₹{totals.get('revenue', 0)}")

        lines.append("\n## Risks & Mitigations")
        lines.append("- Creative fatigue → rotate formats daily; reuse top winners only.")
        lines.append("- Tracking issues → verify UTM and pixels before Day 1.")
        lines.append("- Inventory runouts → sync with ops; set real scarcity only.")

        lines.append("\n## Success Metrics")
        lines.append("- CTR, CPC (paid), follower growth (organic), email CTR/conv, total orders & revenue.")

        return "\n".join(lines)



In [18]:
# -----------------------------
# Demo run
# -----------------------------
if __name__ == "__main__":
    agent = SingleAgent()
    demo_goal = "Plan a 3-day social media promotion for a sports brand with ₹50,000 budget to boost ecommerce sales."
    output = agent.run(demo_goal, brand="FleetAthlete", theme="Speed")

    # Print a compact summary; the full brief is available too.
    print("=== AGENT SUMMARY ===")
    print("Constraints:", output.constraints)
    print("Strategy:", output.strategy)
    print("Top Ideas:", output.ideas[:3])
    print("Forecast:", output.forecast.get("TOTALS"))
    print("\n=== FINAL BRIEF ===")
    print(output.brief)


=== AGENT SUMMARY ===
Constraints: {'budget': 50000.0, 'days': 3}
Strategy: {'allocation': {'Paid Social': 0.5, 'Influencers': 0.2, 'Organic Social': 0.15, 'Email/SMS': 0.15}, 'rupees': {'Paid Social': 25000, 'Influencers': 10000, 'Organic Social': 7500, 'Email/SMS': 7500}, 'days': 3}
Top Ideas: ['Chase your game: form tips (Speed × FleetAthlete)', 'Unlock your game: coach wisdom (Speed × FleetAthlete)', 'Crush your game: micro-workouts (Speed × FleetAthlete)']
Forecast: {'orders': 96, 'revenue': 160061}

=== FINAL BRIEF ===
# Launch Brief

**Goal:** Plan a 3-day social media promotion for a sports brand with ₹50,000 budget to boost ecommerce sales.
**Brand/Theme:** FleetAthlete / Speed
**Constraints:** Budget ₹50000, Duration 3 days

## Channel Strategy & Budget Split
- Paid Social: 50% (₹25000)
- Influencers: 20% (₹10000)
- Organic Social: 15% (₹7500)
- Email/SMS: 15% (₹7500)

## Content Ideas (shortlist)
- Chase your game: form tips (Speed × FleetAthlete)
- Unlock your game: coach w